In [1]:
from pytrends.request import TrendReq
from addon import *
import pandas as pd
import time
import pycountry

In [26]:

pytrends = TrendReq(hl='en-US', tz=360)

## %
kw_list = ["how do i know if i have covid"]
df = pd.DataFrame()

In [27]:
states = [ 'AL', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA', 'IL', 'IN', 'KS', 'KY', 'LA', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

In [4]:
i = 0
while i < len(states):
    try:
        pytrends.build_payload(kw_list, cat=0, timeframe='2020-03-22 2020-10-09', geo="US-" + states[i], gprop='')
        
        state_df = pytrends.interest_over_time()
        state_df["State"] = states[i]
        
        df = pd.concat([df, state_df])
        time.sleep(3)

        print(states[i])
        i += 1

        time.sleep(3)
    except Exception as e:
        i -= 1
        time.sleep(5)
        print(e)


AL
AZ
AR
CA
CO
CT
DE
DC
FL
GA
IL
IN
KS
KY
LA
MD
MA
MI
MN
MS
MO
MT
NE
NV
NH
NJ
NM
NY
NC
ND
OK
OR
PA
RI
SC
SD
TN
TX
VT
VA
WA
WV
WI
WY


In [6]:
df.to_csv("../../data/trends/USA_statewise_2.csv")

In [25]:
df1 = pd.read_pickle("../../data/hopkins_data/df1.pkl")
df1.columns
df1[0] = df1[0] + " 00:00:01"
df1[0] = pd.to_datetime(df1[0])
print(df1)
df1.to_csv("../../data/hopkins_data/df1.csv")

                       0     FIPS         Admin2 Province_State  \
0    2020-03-22 00:00:01  36061.0  New York City       New York   
1    2020-03-22 00:00:01  36059.0         Nassau       New York   
2    2020-03-22 00:00:01  36119.0    Westchester       New York   
3    2020-03-22 00:00:01  36103.0        Suffolk       New York   
4    2020-03-22 00:00:01  36087.0       Rockland       New York   
...                  ...      ...            ...            ...   
2595 2020-05-19 00:00:01  48445.0          Terry          Texas   
2596 2020-05-19 00:00:01  16081.0          Teton          Idaho   
2597 2020-05-19 00:00:01  56039.0          Teton        Wyoming   
2598 2020-05-19 00:00:01  29215.0          Texas       Missouri   
2599 2020-05-19 00:00:01  40139.0          Texas       Oklahoma   

     Country_Region          Last_Update        Lat       Long_  Confirmed  \
0                US        3/22/20 23:45  40.767273  -73.971526       9654   
1                US        3/22/20 23:4